In [1]:
from infoseq.bpe import bpe_from_text

from infoseq.dataset import kjv_path

with open(kjv_path, 'rb') as f:
    kjv = f.read()

len(kjv)

4457889

In [2]:
bpe = bpe_from_text(kjv, max_tokens=200)
len(bpe)

200

In [3]:
[k for k in bpe.encode_map if len(k) > 1][:10]

[b'th', b'e ', b'\r\n', b' t', b' th', b' a', b'nd', b'in', b' h', b't ']

In [4]:
with open('/tmp/bpe.dot', 'w') as f:
    f.write(bpe.to_dot())

In [5]:
print(repr(b"'").replace('"', '\\"'))

b\"'\"
